In [1]:
import numpy as np
import pandas as pd

In [3]:
data = np.load('ROI/trialsLook001.asc.npy',allow_pickle=True)

In [4]:
list_subject = [1]

In [5]:
fix_ = data[0]['fixations']
tuple(fix_[1]['mean'])

(946.9246376811597, 466.97342995169123)

In [6]:
df_ = pd.DataFrame(columns=['Subject', 'Trial', 'Fixation #', 'X_Coord', 'Y_Coord'])

for sub in list_subject:
    for i in range(len(data)):
        subject = sub
        trial = i+1
        fix_ = data[i]['fixations']
        for num, xy in enumerate(fix_):
            coords = tuple(np.round(xy['mean'], 2))
            row = [int(subject), int(trial), int(num+1), coords[0], coords[1]]
            df_.loc[len(df_)] = row

In [8]:
df_

,Subject,Trial,Fixation #,X_Coord,Y_Coord
0,1.0,1.0,1.0,940.17,450.03
1,1.0,1.0,2.0,946.92,466.97
2,1.0,1.0,3.0,944.43,478.19
3,1.0,1.0,4.0,877.94,492.23
4,1.0,1.0,5.0,956.57,422.91
...,...,...,...,...,...
4204,1.0,160.0,27.0,811.87,472.43
4205,1.0,160.0,28.0,998.72,460.75
4206,1.0,160.0,29.0,1023.37,477.28
4207,1.0,160.0,30.0,932.82,481.23


In [10]:
baseline = pd.read_csv('baselines/1.csv')
baseline

,RECORDING_SESSION_LABEL,TRIAL_LABEL,CURRENT_FIX_X,CURRENT_FIX_Y,IS_VALID
0,1,1,929.468403,512.687500,True
1,1,2,917.600000,510.300000,True
2,1,3,931.302500,510.924375,True
3,1,4,923.700000,518.700000,True
4,1,5,923.300000,516.100000,True
...,...,...,...,...,...
155,1,156,889.762963,506.137037,True
156,1,157,895.300000,507.300000,True
157,1,158,895.300000,515.000000,True
158,1,159,903.028834,511.711043,True


In [11]:

x_cross = 1920/2
y_cross = 1080/2

for i in range(len(baseline)):
    trial = baseline.TRIAL_LABEL[i]
    valid = baseline.IS_VALID[i]

    if valid:
        x_base = baseline.CURRENT_FIX_X[i]
        y_base = baseline.CURRENT_FIX_Y[i]

    else:
        x_base = baseline[baseline['IS_VALID']==True]['CURRENT_FIX_X'].mean()
        y_base = baseline[baseline['IS_VALID']==True]['CURRENT_FIX_Y'].mean()

    delta_x, delta_y = (x_cross-x_base), (y_cross-y_base)

    for row in df_[df_['Trial'] == trial].iterrows():
        df_.loc[row[0], 'X_Coord'] = row[1]['X_Coord'] + delta_x
        df_.loc[row[0], 'Y_Coord'] = row[1]['Y_Coord'] + delta_y



In [14]:
df_updated=df_
df_updated

,Subject,Trial,Fixation #,X_Coord,Y_Coord
0,1.0,1.0,1.0,970.701597,477.3425
1,1.0,1.0,2.0,977.451597,494.2825
2,1.0,1.0,3.0,974.961597,505.5025
3,1.0,1.0,4.0,908.471597,519.5425
4,1.0,1.0,5.0,987.101597,450.2225
...,...,...,...,...,...
4204,1.0,160.0,27.0,876.770000,500.4300
4205,1.0,160.0,28.0,1063.620000,488.7500
4206,1.0,160.0,29.0,1088.270000,505.2800
4207,1.0,160.0,30.0,997.720000,509.2300
